# SQLAlchemy

This notebook demonstrates how to load documents from a [CrateDB] database,
using the document loader `CrateDBLoader`, which is based on [SQLAlchemy].

It loads the result of a database query with one document per row.

[CrateDB]: https://github.com/crate/crate
[SQLAlchemy]: https://www.sqlalchemy.org/

## Prerequisites

Install required packages.

In [40]:
#!pip install -r requirements.txt

Populate database.

In [3]:
# Connect to a self-managed CrateDB instance.
CONNECTION_STRING = "crate://crate@localhost/?schema=notebook"

In [4]:
from cratedb_toolkit.util import DatabaseAdapter
from pueblo.io import to_io


def import_mlb_teams_2012():
    """
    Import data into database table `mlb_teams_2012`.

    TODO: Refactor into general purpose package.
    """
    cratedb = DatabaseAdapter(dburi=CONNECTION_STRING)
    url = "https://github.com/crate-workbench/langchain/raw/cratedb/docs/docs/integrations/document_loaders/example_data/mlb_teams_2012.sql"
    with to_io(url) as fp:
        cratedb.run_sql(fp.read())
    cratedb.refresh_table("mlb_teams_2012")


import_mlb_teams_2012()

## Usage

In [5]:
from langchain.document_loaders import CrateDBLoader
from pprint import pprint

loader = CrateDBLoader(
    'SELECT * FROM mlb_teams_2012 ORDER BY "Team" LIMIT 5;',
    url=CONNECTION_STRING,
)
documents = loader.load()

In [6]:
pprint(documents)

[Document(page_content='Team: Angels\nPayroll (millions): 154.49\nWins: 89'),
 Document(page_content='Team: Astros\nPayroll (millions): 60.65\nWins: 55'),
 Document(page_content='Team: Athletics\nPayroll (millions): 55.37\nWins: 94'),
 Document(page_content='Team: Blue Jays\nPayroll (millions): 75.48\nWins: 73'),
 Document(page_content='Team: Braves\nPayroll (millions): 83.31\nWins: 94')]


## Specifying Which Columns are Content vs Metadata

In [15]:
loader = CrateDBLoader(
    'SELECT * FROM mlb_teams_2012 ORDER BY "Team" LIMIT 5;',
    url=CONNECTION_STRING,
    page_content_columns=["Team"],
    metadata_columns=["Payroll (millions)"],
)
documents = loader.load()

In [16]:
pprint(documents)

[Document(page_content='Team: Angels', metadata={'Payroll (millions)': 154.49}),
 Document(page_content='Team: Astros', metadata={'Payroll (millions)': 60.65}),
 Document(page_content='Team: Athletics', metadata={'Payroll (millions)': 55.37}),
 Document(page_content='Team: Blue Jays', metadata={'Payroll (millions)': 75.48}),
 Document(page_content='Team: Braves', metadata={'Payroll (millions)': 83.31})]


## Adding Source to Metadata

In [17]:
loader = CrateDBLoader(
    'SELECT * FROM mlb_teams_2012 ORDER BY "Team" LIMIT 5;',
    url=CONNECTION_STRING,
    source_columns=["Team"],
)
documents = loader.load()

In [18]:
pprint(documents)

[Document(page_content='Team: Angels\nPayroll (millions): 154.49\nWins: 89', metadata={'source': 'Angels'}),
 Document(page_content='Team: Astros\nPayroll (millions): 60.65\nWins: 55', metadata={'source': 'Astros'}),
 Document(page_content='Team: Athletics\nPayroll (millions): 55.37\nWins: 94', metadata={'source': 'Athletics'}),
 Document(page_content='Team: Blue Jays\nPayroll (millions): 75.48\nWins: 73', metadata={'source': 'Blue Jays'}),
 Document(page_content='Team: Braves\nPayroll (millions): 83.31\nWins: 94', metadata={'source': 'Braves'})]
